In [ ]:
from google.colab import drive
drive.mount('\\')

Mounted at /content/drive


In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 40 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 2s (5,572 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 160837 files and d

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, RobustScaler, Normalizer, StandardScaler
from sklearn.model_selection import cross_validate, KFold, train_test_split, cross_val_score
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from sklearn.cluster import KMeans, SpectralClustering
from patsy import dmatrices
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from tqdm import tqdm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
path = '수요예측\\'

In [ ]:
file1 = pd.read_csv(path + '제주도_시간대별_그리드_유동인구_비율포함_20150131.csv', encoding='cp949')
file2 = pd.read_csv(path + '제주도_시간대별_그리드_유동인구_비율포함_20151031.csv', encoding='cp949')
gyukja = pd.read_csv(path + '250m격자기준좌표.csv', encoding='cp949')

In [ ]:
file1.head()

,기준년월,격자 좌표(250m),00시~01시 유동인구,01시~02시 유동인구,02시~03시 유동인구,03시~04시 유동인구,04시~05시 유동인구,05시~06시 유동인구,06시~07시 유동인구,07시~08시 유동인구,08시~09시 유동인구,09시~10시 유동인구,10시~11시 유동인구,11시~12시 유동인구,12시~13시 유동인구,13시~14시 유동인구,14시~15시 유동인구,15시~16시 유동인구,16시~17시 유동인구,17시~18시 유동인구,18시~19시 유동인구,19시~20시 유동인구,20시~21시 유동인구,21시~22시 유동인구,22시~23시 유동인구,23시~00시 유동인구,00시~01시 유동인구 비율,01시~02시 유동인구 비율,02시~03시 유동인구 비율,03시~04시 유동인구 비율,04시~05시 유동인구 비율,05시~06시 유동인구 비율,06시~07시 유동인구 비율,07시~08시 유동인구 비율,08시~09시 유동인구 비율,09시~10시 유동인구 비율,10시~11시 유동인구 비율,11시~12시 유동인구 비율,12시~13시 유동인구 비율,13시~14시 유동인구 비율,14시~15시 유동인구 비율,15시~16시 유동인구 비율,16시~17시 유동인구 비율,17시~18시 유동인구 비율,18시~19시 유동인구 비율,19시~20시 유동인구 비율,20시~21시 유동인구 비율,21시~22시 유동인구 비율,22시~23시 유동인구 비율,23시~00시 유동인구 비율,데이터기준일자
0,2015-01,다다06B00C,194.497419,143.757097,114.303226,86.920645,74.153226,87.136452,130.974194,275.309355,508.254193,779.134516,921.126452,981.390645,967.112903,1005.580000,1003.061613,965.913548,995.931290,1077.305484,1082.694839,854.884839,716.236452,597.364839,388.944516,262.644839,0.005562,0.005618,0.005438,0.005234,0.004642,0.004534,0.004367,0.004781,0.005208,0.005522,0.005543,0.005510,0.005436,0.005651,0.005702,0.005443,0.005449,0.005437,0.005598,0.005784,0.005950,0.005945,0.005597,0.005527,2015-01-31
1,2015-01,다다06C00C,39.689032,29.149677,23.965161,17.630000,14.946452,15.857742,22.868710,46.638065,82.478065,128.218387,153.713871,163.090323,159.608387,163.152581,164.109032,162.646129,168.309032,183.426774,192.222581,152.804839,125.014839,105.290645,74.902903,52.049032,0.001135,0.001139,0.001140,0.001062,0.000936,0.000825,0.000762,0.000810,0.000845,0.000909,0.000925,0.000916,0.000897,0.000917,0.000933,0.000917,0.000921,0.000926,0.000994,0.001034,0.001039,0.001048,0.001078,0.001095,2015-01-31
2,2015-01,다나05B99A,30.085484,21.446452,17.081290,11.660323,10.721290,12.789355,20.000323,41.938065,74.014516,116.520000,143.129355,158.845161,162.468064,163.439677,163.104194,167.342581,172.635161,186.688064,193.322258,148.124516,119.253548,101.770968,69.487419,44.452258,0.000860,0.000838,0.000813,0.000702,0.000671,0.000666,0.000667,0.000728,0.000758,0.000826,0.000861,0.000892,0.000913,0.000918,0.000927,0.000943,0.000945,0.000942,0.000999,0.001002,0.000991,0.001013,0.001000,0.000936,2015-01-31
3,2015-01,다다05B00C,153.049032,111.151290,90.284194,68.203226,59.268710,60.395484,82.628065,155.159032,276.835484,442.083226,541.451935,582.218710,575.083548,588.657742,589.288064,592.700645,617.566774,656.370968,688.528710,557.255807,455.641936,390.546452,283.845806,202.110645,0.004376,0.004343,0.004295,0.004107,0.003710,0.003143,0.002755,0.002694,0.002836,0.003133,0.003258,0.003269,0.003232,0.003308,0.003350,0.003340,0.003379,0.003312,0.003560,0.003770,0.003785,0.003887,0.004084,0.004253,2015-01-31
4,2015-01,다다05C00D,59.621935,43.564194,35.652581,26.703871,22.887419,23.128387,31.820000,62.128065,110.730968,174.586774,211.708710,226.140000,222.253871,226.438710,227.062581,227.711613,237.304839,256.556129,272.153548,216.783226,175.845161,150.011613,109.524194,77.792903,0.001705,0.001702,0.001696,0.001608,0.001433,0.001204,0.001061,0.001079,0.001135,0.001237,0.001274,0.001270,0.001249,0.001272,0.001291,0.001283,0.001298,0.001295,0.001407,0.001467,0.001461,0.001493,0.001576,0.001637,2015-01-31


In [ ]:
gyukja.head()

,가로좌표 (X Coord),세로좌표 (Y Coord),국가지점좌표계250키(SPO_NO_250)
0,884875.0001,1458875,나나84D58D
1,885125.0001,1458875,나나85D58A
2,884875.0001,1459125,나나84A59D
3,885125.0001,1459125,나나85A59A
4,884875.0001,1459375,나나84B59D


In [ ]:
file1.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
00시~01시 유동인구,13365.0,2.616616,13.960913,0.000000e+00,0.073226,0.222903,0.756452,432.689677
01시~02시 유동인구,13365.0,1.914730,10.328992,0.000000e+00,0.055806,0.170000,0.566774,331.789677
02시~03시 유동인구,13365.0,1.572813,8.300213,0.000000e+00,0.051290,0.153226,0.504839,272.238710
03시~04시 유동인구,13365.0,1.242619,6.299816,0.000000e+00,0.042903,0.128710,0.428387,198.585806
04시~05시 유동인구,13365.0,1.195195,5.544954,0.000000e+00,0.048065,0.140323,0.456129,156.502258
05시~06시 유동인구,13365.0,1.437865,6.031205,0.000000e+00,0.064194,0.188710,0.599677,135.499677
06시~07시 유동인구,13365.0,2.244195,8.525009,0.000000e+00,0.121290,0.346774,1.066774,155.104194
07시~08시 유동인구,13365.0,4.308566,15.592805,0.000000e+00,0.250000,0.736129,2.222581,304.115161
08시~09시 유동인구,13365.0,7.302535,26.595010,3.548387e-03,0.429032,1.276129,3.768387,514.837097
09시~10시 유동인구,13365.0,10.556918,39.830660,5.483871e-03,0.614839,1.774194,5.284194,779.134516


In [ ]:
file1.describe().transpose()['mean'].sort_values()

02시~03시 유동인구 비율     0.000075
06시~07시 유동인구 비율     0.000075
03시~04시 유동인구 비율     0.000075
01시~02시 유동인구 비율     0.000075
04시~05시 유동인구 비율     0.000075
13시~14시 유동인구 비율     0.000075
07시~08시 유동인구 비율     0.000075
00시~01시 유동인구 비율     0.000075
15시~16시 유동인구 비율     0.000075
16시~17시 유동인구 비율     0.000075
11시~12시 유동인구 비율     0.000075
20시~21시 유동인구 비율     0.000075
12시~13시 유동인구 비율     0.000075
18시~19시 유동인구 비율     0.000075
08시~09시 유동인구 비율     0.000075
21시~22시 유동인구 비율     0.000075
23시~00시 유동인구 비율     0.000075
09시~10시 유동인구 비율     0.000075
14시~15시 유동인구 비율     0.000075
22시~23시 유동인구 비율     0.000075
05시~06시 유동인구 비율     0.000075
10시~11시 유동인구 비율     0.000075
19시~20시 유동인구 비율     0.000075
17시~18시 유동인구 비율     0.000075
04시~05시 유동인구        1.195195
03시~04시 유동인구        1.242619
05시~06시 유동인구        1.437865
02시~03시 유동인구        1.572813
01시~02시 유동인구        1.914730
06시~07시 유동인구        2.244195
00시~01시 유동인구        2.616616
23시~00시 유동인구        3.555340
07시~08시 유동인구        4.308566
22시~23시 유동인구        5.199883
08시~09시 유동인구  

In [ ]:
file1 = file1.loc[:, ['격자 좌표(250m)', '14시~15시 유동인구']]
file1.rename(columns={'격자 좌표(250m)': '격자좌표', '14시~15시 유동인구':'0131유동인구'}, inplace=True)

In [ ]:
file1

,격자좌표,0131유동인구
0,다다06B00C,1003.061613
1,다다06C00C,164.109032
2,다나05B99A,163.104194
3,다다05B00C,589.288064
4,다다05C00D,227.062581
...,...,...
13360,다다28B00B,0.386452
13361,다다48D01A,0.091613
13362,다나20C98A,0.054194
13363,다다48C00B,0.079677


In [ ]:
file2 = file2.loc[:, ['지번코드', '14시~15시 유동인구']]
file2.rename(columns={'지번코드': '격자좌표', '14시~15시 유동인구':'1031유동인구'}, inplace=True)

In [ ]:
file2

,격자좌표,1031유동인구
0,다다10C02A,153.36
1,다다09A03C,196.37
2,다다09D02C,395.62
3,다다09D02D,185.97
4,다다09C02C,424.63
...,...,...
16681,나나91B72B,0.50
16682,나나90C74A,0.81
16683,나나87D71A,0.76
16684,나나94B74C,0.66


In [ ]:
dataset = pd.merge(file1, file2, on='격자좌표')

In [ ]:
dataset

,격자좌표,0131유동인구,1031유동인구
0,다다06B00C,1003.061613,572.16
1,다다06C00C,164.109032,158.71
2,다나05B99A,163.104194,225.12
3,다다05B00C,589.288064,382.08
4,다다05C00D,227.062581,162.91
...,...,...,...
13319,다다28B00B,0.386452,0.34
13320,다다48D01A,0.091613,1.76
13321,다나20C98A,0.054194,0.04
13322,다다48C00B,0.079677,0.12


In [ ]:
gyukja.rename(columns={'가로좌표 (X Coord)': '가로좌표', '세로좌표 (Y Coord)':'세로좌표', '국가지점좌표계250키(SPO_NO_250)':'격자좌표'}, inplace=True)

In [ ]:
dataset2 = pd.merge(dataset, gyukja, on='격자좌표')

In [ ]:
dataset2.drop('격자좌표', axis=1, inplace=True)

In [ ]:
dataset2

,0131유동인구,1031유동인구,가로좌표,세로좌표
0,1003.061613,572.16,906625.0001,1500375
1,164.109032,158.71,906625.0001,1500625
2,163.104194,225.12,905125.0001,1499375
3,589.288064,382.08,905625.0001,1500375
4,227.062581,162.91,905875.0001,1500625
...,...,...,...,...
13319,0.386452,0.34,928375.0001,1500375
13320,0.091613,1.76,948125.0001,1501875
13321,0.054194,0.04,920125.0001,1498625
13322,0.079677,0.12,948375.0001,1500625


In [ ]:
!pip install pyproj

ERROR: Could not find a version that satisfies the requirement Porj (from versions: none)
ERROR: No matching distribution found for Porj


In [ ]:
import folium as f
from pyproj import Proj, transform

In [ ]:
# Projection 정의
# 중부원점(Bessel): 서울 등 중부지역 EPSG:2097
proj_1 = Proj(init='epsg:5178')

# WGS84 경위도: GPS가 사용하는 좌표계 EPSG:4326
proj_2 = Proj(init='epsg:4326')

/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:306: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axi

In [ ]:
DataFrame = dataset2.copy()

x_list = []
y_list = []

for idx, row in DataFrame.iterrows():
    x, y = row['가로좌표'], row['세로좌표']
    x_, y_ = transform(proj_1, proj_2, x, y)
    x_list.append(x_)
    y_list.append(y_)
    
dataset2['lon'] = x_list
dataset2['lat'] = y_list

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  


In [ ]:
dataset2

,0131유동인구,1031유동인구,가로좌표,세로좌표,lon,lat
0,1003.061613,572.16,906625.0001,1500375,126.492711,33.493925
1,164.109032,158.71,906625.0001,1500625,126.492685,33.496180
2,163.104194,225.12,905125.0001,1499375,126.476672,33.484775
3,589.288064,382.08,905625.0001,1500375,126.481948,33.493837
4,227.062581,162.91,905875.0001,1500625,126.484612,33.496114
...,...,...,...,...,...,...
13319,0.386452,0.34,928375.0001,1500375,126.726827,33.495604
13320,0.091613,1.76,948125.0001,1501875,126.939343,33.510274
13321,0.054194,0.04,920125.0001,1498625,126.638178,33.479237
13322,0.079677,0.12,948375.0001,1500625,126.942107,33.499012


In [ ]:
x_list = []
y_list = []

for idx, row in DataFrame[:5].iterrows():
    x, y = row['세로좌표'], row['가로좌표']
    x_, y_ = transform(proj_1, proj_2, x, y)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  


In [ ]:
dataset2.to_csv(path + '1501_1510_위경도별유동인구.csv', index=False)

In [ ]:
dataset2

,0131유동인구,1031유동인구,가로좌표,세로좌표,lon,lat
0,1003.061613,572.16,906625.0001,1500375,126.492711,33.493925
1,164.109032,158.71,906625.0001,1500625,126.492685,33.496180
2,163.104194,225.12,905125.0001,1499375,126.476672,33.484775
3,589.288064,382.08,905625.0001,1500375,126.481948,33.493837
4,227.062581,162.91,905875.0001,1500625,126.484612,33.496114
...,...,...,...,...,...,...
13319,0.386452,0.34,928375.0001,1500375,126.726827,33.495604
13320,0.091613,1.76,948125.0001,1501875,126.939343,33.510274
13321,0.054194,0.04,920125.0001,1498625,126.638178,33.479237
13322,0.079677,0.12,948375.0001,1500625,126.942107,33.499012


In [ ]:
df = dataset2.copy()

In [ ]:
df.sort_values(by='0131유동인구', ascending=False).reset_index(inplace=True)

In [ ]:
df[:20]

,0131유동인구,1031유동인구,가로좌표,세로좌표,lon,lat
0,1003.061613,572.16,906625.0001,1500375,126.492711,33.493925
1,164.109032,158.71,906625.0001,1500625,126.492685,33.496180
2,163.104194,225.12,905125.0001,1499375,126.476672,33.484775
3,589.288064,382.08,905625.0001,1500375,126.481948,33.493837
4,227.062581,162.91,905875.0001,1500625,126.484612,33.496114
5,496.523871,395.20,906375.0001,1500375,126.490020,33.493903
6,165.781613,152.67,908625.0001,1502375,126.514034,33.512134
7,69.073548,24.88,908375.0001,1503125,126.511266,33.518877
8,90.465484,66.27,907375.0001,1503125,126.500499,33.518790
9,85.765484,97.61,908125.0001,1502125,126.508677,33.509837


In [ ]:
lat_c = 33.493925
lon_c = 126.492711

m = folium.Map(location=[lat_c, lon_c], zoom_start=11)

for idx, row in df[:20].iterrows():

    lat_ = row['lat']
    lon_ = row['lon']

    folium.Marker(location=[lat_, lon_],radical=10).add_to(m)

In [ ]:
path = '데이터\\'

In [ ]:
file1 = pd.read_excel(path + 'DATA_1.xlsx')
# file2 = pd.read_csv(path + '제주도_시간대별_그리드_유동인구_비율포함_20151031.csv', encoding='cp949')

In [ ]:
file1.drop(['지역'], axis=1, inplace=True)

In [ ]:
file1.rename(columns={'2021-01-01 00:00:00':'2020년 1월 방문객 수'}, inplace=True)

In [ ]:
file1.dropna(inplace=True)

In [ ]:
file1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43 entries, 0 to 43
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   지역명             43 non-null     object 
 1   맛집수             43 non-null     int64  
 2   관광지 수           43 non-null     int64  
 3   숙박업소수           43 non-null     int64  
 4   버스정류장 수         43 non-null     int64  
 5   해수욕장유무          43 non-null     int64  
 6   2020년 1월 방문객 수  43 non-null     float64
 7   2021년 1월 방문객 수  43 non-null     float64
dtypes: float64(2), int64(5), object(1)
memory usage: 3.0+ KB


In [ ]:
file1.set_index('지역명', inplace=True)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [ ]:
file1.head()

,맛집수,관광지 수,숙박업소수,버스정류장 수,해수욕장유무,2020년 1월 방문객 수,2021년 1월 방문객 수
지역명,,,,,,,
한림읍,7,23,65,19874,1,408817.0,245540.0
애월읍,17,25,113,52414,2,734535.0,442606.0
구좌읍,7,28,32,23134,5,438797.0,245380.0
조천읍,26,24,58,45480,1,566892.0,278139.0
한경면,2,14,11,0,1,176047.0,107564.0


In [ ]:
df = file1.copy()

In [ ]:
col_list = df.columns.tolist()

In [ ]:
sc = StandardScaler()
df[col_list] = sc.fit_transform(df[col_list])

In [ ]:
df.head()

In [ ]:
cols = list(df.columns) 
cols 

x_col = cols[:5] 
y_col = cols[6] 
x_col  
y_col 

In [ ]:
train, test = train_test_split(df, test_size = 0.2, random_state=42) 

train.shape 
test.shape

(9, 7)

In [ ]:
model = LinearRegression() 
model.fit(X = train[x_col], y = train[y_col])




LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
model.coef_    

array([ 0.13227012,  0.3985421 ,  0.18785542,  0.25773582, -0.14794459])

In [ ]:
model.intercept_ 

0.007001797049292269

In [ ]:
y_pred = model.predict(X=test[x_col]) 
y_pred 
len(y_pred)


Y = test[y_col] 
Y 
len(Y)

9

In [ ]:
mse = mean_squared_error(y_true = Y, y_pred = y_pred) 
print('mse =', mse) 

mse = 0.18493997338949558


In [ ]:
import seaborn as sns

from ipywidgets import interact
def lm_plot(x):
    sns.lmplot(x="버스정류장 수",
               y="2021년 1월 방문객 수",
               data=df,
               order = x,
               line_kws={'color':"red"}, scatter_kws={'edgecolor':"white"})
    
interact(lm_plot, x=2)

interactive(children=(IntSlider(value=2, description='x', max=6, min=-2), Output()), _dom_classes=('widget-int…

<function __main__.lm_plot>

In [ ]:
cols = list(df.columns) 
cols 

x_col = cols[:4] 
y_col = cols[6] 
x_col  
y_col 

'2021년 1월 방문객 수'

In [ ]:
train, test = train_test_split(df, test_size = 0.2, random_state=42) 

train.shape 
test.shape

(9, 7)

In [ ]:
model = LinearRegression() 
model.fit(X = train[x_col], y = train[y_col])

model.coef_    

array([0.11649801, 0.29867698, 0.17433582, 0.28172223])

In [ ]:
model.intercept_ 

0.0010555726987038078

In [ ]:
y_pred = model.predict(X=test[x_col]) 
y_pred 
len(y_pred)


Y = test[y_col] 
Y 
len(Y)

9

In [ ]:
mse = mean_squared_error(y_true = Y, y_pred = y_pred) 
print('mse =', mse) 

mse = 0.17506134926280523
